In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os

batch_size = 4
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
### my data dir
data_dir = 'Data2'


2024-03-26 23:12:33.266204: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-26 23:12:33.266241: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-26 23:12:33.266275: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-26 23:12:33.272288: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/tarush/.local/lib/python3.10/site-packages/m

In [2]:
### we will use datagenerator to load from directory and create train validation and test sets
from keras.preprocessing.image import ImageDataGenerator


def preprocess_input(image):
    image = tf.keras.applications.mobilenet.preprocess_input(image)
    return image

# Define the data directories and the target image size
target_size = (224, 224)

# Create an ImageDataGenerator object with preprocessing
datagen = ImageDataGenerator(
    validation_split=0.2,  # 20% of data will be used for validation
    rescale=1./255,  # Rescale pixel values to [0, 1]
    preprocessing_function=preprocess_input  # Preprocessing function to be applied
)

# Create a training data generator
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=target_size,
    batch_size=8,
    class_mode='categorical',
    subset='training',  # Set as training data
)

# Create a validation data generator
validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=target_size,
    batch_size=32,
    class_mode='categorical',
    subset='validation',  # Set as validation data
)


print(train_generator.class_indices)

Found 3859 images belonging to 94 classes.
Found 924 images belonging to 94 classes.
{'Alexandrite': 0, 'Almandine': 1, 'Amazonite': 2, 'Amber': 3, 'Amethyst': 4, 'Ametrine': 5, 'Andalusite': 6, 'Andradite': 7, 'Aquamarine': 8, 'Aventurine Green': 9, 'Aventurine Yellow': 10, 'Basalt': 11, 'Benitoite': 12, 'Beryl Golden': 13, 'Bixbite': 14, 'Bloodstone': 15, 'Blue Lace Agate': 16, 'Carnelian': 17, 'Cats Eye': 18, 'Chalcedony': 19, 'Chalcedony Blue': 20, 'Chrome Diopside': 21, 'Chrysoberyl': 22, 'Chrysocolla': 23, 'Chrysoprase': 24, 'Citrine': 25, 'Coal': 26, 'Coral': 27, 'Danburite': 28, 'Diamond': 29, 'Diaspore': 30, 'Dumortierite': 31, 'Emerald': 32, 'Fluorite': 33, 'Garnet Red': 34, 'Goshenite': 35, 'Granite': 36, 'Grossular': 37, 'Hessonite': 38, 'Hiddenite': 39, 'Iolite': 40, 'Jade': 41, 'Jasper': 42, 'Kunzite': 43, 'Kyanite': 44, 'Labradorite': 45, 'Lapis Lazuli': 46, 'Larimar': 47, 'Limestone': 48, 'Malachite': 49, 'Marble': 50, 'Moonstone': 51, 'Morganite': 52, 'Onyx Black': 53,

In [3]:
### here we will be using vit as out base model and we will add a few layers on top of it help with the classification
from vit_keras import  vit, utils
from keras.layers import Flatten, Dense, Dropout
from keras.models import Sequential

'''base_model = vit.vit_b16(image_size=target_size[0], activation='softmax', pretrained=True)
model = Sequential([
    base_model,
    Flatten(),
    Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    Dropout(0.5),
    Dense(94, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()
'''


model2 = tf.keras.models.Sequential([
    tf.keras.layers.experimental.preprocessing.Rescaling(1./255),
    tf.keras.layers.Conv2D(512, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(256, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(128, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(94, activation='softmax')
])

model2.compile(optimizer='adagrad', loss='categorical_crossentropy', metrics=['accuracy'])

history2 = model2.fit(
      train_generator,
      steps_per_epoch=train_generator.samples // batch_size,
      epochs=40,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples // batch_size,
      callbacks=[early_stopping, rampupLR])

/home/tarush/.local/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
2024-03-26 23:12:35.410331: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-26 23:12:35.414770: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from Sy

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vit-b16 (Functional)        (None, 1000)              86567656  
                                                                 
 flatten (Flatten)           (None, 1000)              0         
                                                                 
 dense (Dense)               (None, 512)               512512    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 94)                48222     
                                                                 
Total params: 87128390 (332.37 MB)
Trainable params: 87128390 (332.37 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [4]:
### to this model we will be passing and fitting our dataset
from keras.callbacks import EarlyStopping, LearningRateScheduler
# Early stopping callback
early_stopping = EarlyStopping(patience=3, restore_best_weights=True)
epochs = 20  # Adjust the number of epochs
def step_decay(epoch):
    initial_lr = 0.001  # Initial learning rate
    drop = 0.5  # Learning rate drop factor
    epochs_drop = 5  # Number of epochs after which learning rate will drop
    new_lr = initial_lr * (drop ** (epoch // epochs_drop))
    return new_lr


learnRateSch = LearningRateScheduler(step_decay)

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // 8,  # Use the same batch size as in your generator
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // 32,  # Use the same batch size as in your generator
    epochs=10
)

Epoch 1/10


2024-03-26 23:12:52.821747: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8902
2024-03-26 23:12:52.883790: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-03-26 23:12:53.088764: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-03-26 23:12:54.115350: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x76e4880fcc90 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-26 23:12:54.115399: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2024-03-26 23:12:54.119871: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-03-26 23:12:54.192873: I ./tensorflow/compiler/jit/

482/482 [==============================] - 150s 262ms/step - loss: 4.4188 - accuracy: 0.0769 - val_loss: 4.0496 - val_accuracy: 0.0893
Epoch 2/10
482/482 [==============================] - 134s 278ms/step - loss: 4.0993 - accuracy: 0.0821 - val_loss: 4.0527 - val_accuracy: 0.0882
Epoch 3/10
398/482 [=======================>......] - ETA: 22s - loss: 4.0941 - accuracy: 0.0827

KeyboardInterrupt: 